In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Mon Feb 24 11:58:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:01:00.0 Off |                  Off |
| 42%   66C    P2            160W /  230W |    2941MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
import os

if 'notebooks' in os.getcwd():
    os.chdir("..")


from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import time
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import aiohttp
import asyncio
from tqdm.asyncio import tqdm
import torch
import scienceplots
plt.style.use(['science', 'no-latex'])
from IPython.display import clear_output

from src.text_utils import trim_text_to_token_limit
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from langdetect import detect

tqdm.pandas()

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", cache_dir = '/Data')

In [5]:
torch.cuda.set_per_process_memory_fraction(0.6, device=0)


In [6]:
base_data = pd.read_json("data/wikipedia_dataset.json")

## Loading data

In [7]:
generated_df = pd.read_pickle("data/generated_dataset_Qwen2.5-0.5B-Instruct.pkl")

In [15]:
generated_df['language'] = generated_df['generated_text'].progress_apply(detect)

  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 5000/5000 [00:13<00:00, 368.15it/s]


In [22]:
mask = generated_df['language'] == 'en'
generated_df[mask]

,id,generated_text,language
16,230018,Joseph Priestley (1733-1804) was a British the...,en
110,13261,O rock é um gênero musical que se desenvolveu ...,en
327,1668,Rembrandt van Rijn (1606-1669) was a Dutch pai...,en
337,9019,John Locke (1632-1704) was a British philosoph...,en
361,48024,Johannes Kepler (1571-1630) was a German astro...,en
377,19175,O Celtic Frost foi uma banda de metal formada ...,en
567,7632,Adam Smith (1723-1790) was a Scottish philosop...,en
612,35448,"Sir James Paul McCartney, conhecido como McCar...",en
613,5571703,A série Stranger Things é uma série de televis...,en
615,9902,O Red Hot Chili Peppers é uma banda de rock do...,en


In [27]:
print(base_data.query("id == 230018")['text'].item())

Joseph Priestley (Birstall,  (Velho estilo) – Northumberland, ) foi um teólogo, clérigo dissidente, filósofo natural, educador, teórico e político britânico que publicou mais de 150 obras. A ele normalmente é creditada a descoberta do oxigênio, apesar de Carl Wilhelm Scheele e Antoine Lavoisier também a reivindicarem, por ter Priestley escondido a descoberta do novo gás.

Durante sua vida, a considerável reputação científica de Priestley baseou-se em seu invento da "água carbonatada", seus escritos sobre a electricidade, e sua descoberta de vários "ares" (gases), sendo a mais famosa dentre suas descobertas o "ar deflogisticado" (oxigênio). No entanto, sua determinação em defender a Teoria do flogisto para rejeitar o que passaria a ser a Revolução Química, eventualmente deixava o invento oculto no interior da comunidade científica.

A ciência de Priestley foi sempre associada a sua teologia, e ele tentou consistentemente unir racionalismo iluminista com o teísmo cristão. Em seus textos 

In [23]:
print(generated_df[mask].iloc[0]['generated_text'])

Joseph Priestley (1733-1804) was a British theologian, philosopher, educator, and politician who was a key figure in the development of the field of natural philosophy. He was a prolific writer, and his works include "Observations on Man" (1749), "The Rudiments of English Grammar" (1761), and "The History and Present State of Electricity" (1768). Priestley's influence on the field of natural philosophy was significant, and he was a key figure in the development of the utilitarian philosophy of the Enlightenment. He was a member of the Royal Society and a founding member of the Royal Institute.<|im_end|>
